# <center>Laboratorium Analiza i bazy danych </center>

## <center>Łączenie tabel, podzapytania i funkcje agregujące</center>

## Przykładowe tabele obrazujące łączenie

Do zobrazowania operacji łączenia zostaną użyte tabele:

```sql
CREATE TABLE shape_a (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
 
CREATE TABLE shape_b (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
```
 
Polecenie CREATE TABLE tworzy tabelę o zadanej nazwie i strukturze. Ogólna postać to:
```sql
CREATE TABLE tab_name (
    col_name1 data_type constrain,
    col_name1 data_type constrain,
    ...
);
```
Należy uzupełnić ją danymi:
```sql
INSERT INTO shape_a (id, shape)
VALUES
    (1, 'Trójkąt'),
    (2, 'Kwadrat'),
    (3, 'Deltoid'),
    (4, 'Traper');
 
INSERT INTO shape_b (id, shape)
VALUES
    (1, 'Kwadrat'),
    (2, 'Trójkąt'),
    (3, 'Romb'),
    (4, 'Równoległobok');
```
Komenda INSERT INTO pozwala na dodanie do tabeli rekordów. Ogólna postać to:

```sql
INSERT INTO tab_name (col1_name, col2_name2, ...) 
VALUES
    (val1_col1, val2_col2),
    (val2_col1, val2_col2),
    ...
```

## Inner join 

Jest to podstawowy rodzaj złączenie. Ten sposób złączenia wybiera  te wiersze, dla których warunek złączenia jest spełniony. W żadnej z łączonych tabel kolumna użyta do łączenia nie może mieć wartości NULL. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
INNER JOIN shape_b b ON a.shape = b.shape;
```
W zapytaniu powyżej użyto *aliasów* nazw tabel i column wynikowych, jest to szczególnie przydatne przy długich nazwach tabel i wprowadza czytelność w zapytaniu.

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|

## OUTER JOIN

Istnieją trzy rodzaje złączeń OUTER:
- LEFT OUTER JOIN,
- RIGHT OUTER JOIN,
- FULL OUTER JOIN.

### LEFT OUTER JOIN

Ten rodzaj złączenie zwróci wszystkie rekordy z lewej tablicy i dopasuje do nich rekordy z prawej tablicy które spełniją zadany warunek złączenia. Jeżeli w prawej tablicy nie występują rekordy spełnijące warunek złączenia z lewą w ich miejscu pojawią się wartości NULL.

#### Przykład 1:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
LEFT JOIN shape_b b ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid|NULL|NULL|
|4|Traper|NULL|NULL|

#### Przykład 2:
```sql
SELECT
    b.id id_b,
    b.shape shape_b,
    a.id id_a,
    a.shape shape_a   
FROM
    shape_b b
LEFT JOIN shape_a a ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Kwadrat|2|Kwadrat|
|2|Trójkąt|1|Trójkąt|
|3|Romb|NULL|NULL|
|4|Równoległobok|NULL|NULL|

### RIGHT OUTER JOIN

Działa jak left outer join z tym, że prawa tablica w zapytaniu jest brana w całości.

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
RIGHT JOIN shape_b b ON a.shape = b.shape;
```

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|2|Kwadrat|1|Kwadrat|
|1|Trójkąt|2|Trójkąt|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|


### FULL OUTER JOIN

Jest złączeniem które zwraca:
- wiersze dla których warunek złączenia jest spełniony,
- wiersze z lewej tabeli dla których nie ma odpowiedników w prawej,
- wiersze z prawej tabeli dla których nie ma odpowiedników w lewej. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
FULL JOIN shape_b b ON a.shape = b.shape;
```
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid"|NULL|NULL|
|4|Traper|NULL|NULL|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|

## Podzapytania

Podzapytanie zagnieżdżone SELECT znajduje się wewnątrz zewnętrznego zapytania SELECT, np. po klauzuli WHERE, HAVING lub FROM. W przypadku tego rodzaju zapytań w pierwszej kolejności wykonywane są wewnętrzne zapytania SELECT, a ich wynik jest wykorzystywany do zewnętrznego zapytania SELECT. Stąd łatwo zuważyć, że mogą one służyć do poprawy wydajności obsługi zapytania. Należy dobierać podzapytania tak by najbardziej zagnieżdżone podzapytanie zawierało najmniejszy zbiór poszukiwań. 

#### Przykład:
Jeżeli chcemy znaleźć w bazie informację o tytułach filmów zwróconych w zadanym okresie możemy wykonać następujące zapytanie:
```sql
SELECT
   film_id,
   title
FROM
   film
WHERE
   film_id IN (
      SELECT
         inventory.film_id
      FROM
         rental
      INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
      WHERE
         return_date BETWEEN '2005-05-29'
      AND '2005-05-30'
   );
```

#### Wynik
|film_id|title|
|-|-|
|307|Fellowship Autumn|
|255|Driving Polish|
|388|Gunfight Moon|
|130|Celebrity Horn|
|563|Massacre Usual|
|397|Hanky October|
|...|...|

### Używanie podzapytań

Pod zapytania mogą być używane w :
- SELECT,
- UPDATE,
- DELETE,
- Funkcjach agregujących,
- Do definiowania tabel tymczasowych.

Używając podzapytań zapytania SQL szybko mogą stać się mało czytelne. Przez co będą trudne w zrozumieniu i późniejszym utrzymaniu. W celu analizy zapytań można użyć klauzuli __EXPLAIN__, która przeanalizuje zapytanie. Klauzula ta może służyć również do porównywania wydajności zapytań

#### Przykład:
```sql
EXPLAIN SELECT
   *
FROM
   film
```

## Funkcje agregujące

Funkcje agregujące wykonują obliczenia na zestawie wierszy i zwracają pojedynczy wiersz. PostgreSQL udostępnia wszystkie standardowe funkcje agregujące SQL w następujący sposób:
- AVG () - zwraca średnią wartość.
- COUNT () - zwraca liczbę wartości.
- MAX () - zwraca maksymalną wartość.
- MIN () - zwraca minimalną wartość.
- SUM () - zwraca sumę wszystkich lub różnych wartości.

Pełna lista funkcji agregującej: https://www.postgresql.org/docs/9.5/functions-aggregate.html

Często używamy funkcji agregujących z klauzulą GROUP BY w instrukcji SELECT. W tych przypadkach klauzula GROUP BY dzieli zestaw wyników na grupy wierszy i funkcja agregująca wykonuje obliczenia dla każdej grupy, np. maksimum, minimum, średnia itp. Funkcji agregujących można używać funkcji agregujących jako wyrażeń tylko w następujących klauzulach: SELECT i HAVING.

### GROUP BY
Klauzula GROUP BY dzieli wiersze zwrócone z instrukcji SELECT na grupy. Dla  każdej grupy można zastosować funkcję agregującą, np. SUM aby obliczyć sumę pozycji lub
COUNT aby uzyskać liczbę elementów w grupach.

Poniższa instrukcja ilustruje składnię klauzuli GROUP BY:
```sql
SELECT 
    column_1, 
    aggregate_function(column_2)
FROM 
    tbl_name
GROUP BY 
    column_1;
```
Klauzula GROUP BY musi pojawić się zaraz po klauzuli FROM lub WHERE, n0astępnie GROUP BY zawiera listę  kolumna oddzielonych przecinkami. 

### HAVING
Często używamy klauzuli HAVING w połączeniu z klauzulą GROUP BY do filtrowania wierszy grup
które nie spełniają określonego warunku.

Poniższa instrukcja ilustruje typową składnię klauzuli HAVING:
```sql
SELECT
    column_1,
    aggregate_function (column_2)
FROM
    tbl_name
GROUP BY
    column_1
HAVING
    condition;
```
Klauzula HAVING ustawia warunek dla wierszy grup utworzonych przez klauzulę GROUP BY.  

Klauzula GROUP BY ma zastosowanie, podczas gdy klauzula WHERE określa wcześniej warunki dla poszczególnych wierszy.

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS:  
  
1. Znajdź listę wszystkich filmów o tej samej długości.
2. Znajdź wszystkich klientów mieszkających w tym samym mieście.
3. Oblicz średni koszt wypożyczenia wszystkich filmów.
4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
8. Oblicz średnią wartość wypożyczenia filmu.
9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

In [9]:
import main
import pandas as pd
import psycopg2 as pg
connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb_2023', user='wbauer_adb', password='adb2020')

In [10]:
#zadanie 1 
wynik_zapyt_1 = main.number_film_by_length()
wynik_zapyt_1

c:\Piotr.K\AGH_STUDIA\semV\Analiza_i_Bazy_Danych\Lab_6\laboratorium-join-agr-sub-piotr0202\main.py:72: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT film.length, COUNT(film.film_id) from film WHERE film.length >= %s AND film.length <= %s\


,length,count
0,46,5
1,47,7
2,48,11
3,49,5
4,50,9
...,...,...
135,181,10
136,182,6
137,183,5
138,184,8


In [23]:
#zadanie 2. Znajdź wszystkich klientów mieszkających w tym samym mieście.

df_zad2 = pd.read_sql('SELECT city.city, customer.first_name, customer.last_name FROM city\
                JOIN address ON address.city_id = city.city_id\
                JOIN customer ON customer.address_id = address.address_id\
                ORDER BY city.city, customer.last_name, customer.first_name;',con=connection)
df_zad2


C:\Users\piotr\AppData\Local\Temp\ipykernel_2404\3434821580.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_zad2 = pd.read_sql('SELECT city.city, customer.first_name, customer.last_name FROM city\


,city,first_name,last_name
0,A Corua (La Corua),Julie,Sanchez
1,Abha,Peggy,Myers
2,Abu Dhabi,Tom,Milner
3,Acua,Glen,Talbert
4,Adana,Larry,Thrasher
...,...,...,...
594,Zaria,Constance,Reid
595,Zeleznogorsk,Jack,Foust
596,Zhezqazghan,Byron,Box
597,Zhoushan,Guy,Brownlee


In [27]:
#3. Oblicz średni koszt wypożyczenia wszystkich filmów.
df_zad3 = pd.read_sql('SELECT AVG(payment.amount)  FROM payment',con=connection)
df_zad3

C:\Users\piotr\AppData\Local\Temp\ipykernel_2404\372512941.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_zad3 = pd.read_sql('SELECT AVG(payment.amount)  FROM payment',con=connection)


,avg
0,4.200606


In [37]:
#4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
df_zad4 = pd.read_sql(' SELECT category.name AS category_name, COUNT(film.film_id) AS film_count\
                        FROM category\
                        JOIN film_category ON category.category_id = film_category.category_id\
                        JOIN film ON film.film_id = film_category.film_id\
                        GROUP BY category.name\
                        ORDER BY category.name;',con=connection)
df_zad4

C:\Users\piotr\AppData\Local\Temp\ipykernel_2404\3505429977.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_zad4 = pd.read_sql(' SELECT category.name AS category_name, COUNT(film.film_id) AS film_count\


,category_name,film_count
0,Action,64
1,Animation,66
2,Children,60
3,Classics,57
4,Comedy,58
5,Documentary,68
6,Drama,62
7,Family,69
8,Foreign,73
9,Games,61


In [40]:
#5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
df_zad5 = pd.read_sql(' SELECT country.country AS country, COUNT(customer.customer_id) AS customer_count\
                        FROM country\
                        JOIN city ON city.country_id = country.country_id\
                        JOIN address ON address.city_id = city.city_id\
                        JOIN customer ON customer.address_id = address.address_id\
                        GROUP BY country.country\
                        ORDER BY country.country;',con=connection)
df_zad5

C:\Users\piotr\AppData\Local\Temp\ipykernel_2404\513476526.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_zad5 = pd.read_sql(' SELECT country.country AS country, COUNT(customer.customer_id) AS customer_count\


,country,customer_count
0,Afghanistan,1
1,Algeria,3
2,American Samoa,1
3,Angola,2
4,Anguilla,1
...,...,...
103,Vietnam,6
104,"Virgin Islands, U.S.",1
105,Yemen,4
106,Yugoslavia,2


In [51]:
#6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
df_zad6 = pd.read_sql(' SELECT store.store_id AS store_id, store.manager_staff_id AS manager_id, store.address_id as store_address\
                        FROM store\
                        JOIN address ON address.address_id = store.address_id\
                        JOIN customer ON customer.address_id = address.address_id\
                        GROUP BY store.store_id, store.manager_staff_id, store.address_id\
                        HAVING COUNT(customer.customer_id) > 100 AND COUNT(customer.customer_id) < 300',con=connection)
df_zad6

C:\Users\piotr\AppData\Local\Temp\ipykernel_2404\3397308030.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_zad6 = pd.read_sql(' SELECT store.store_id AS store_id, store.manager_staff_id AS manager_id, store.address_id as store_address\


,store_id,manager_id,store_address


In [54]:
#7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
df_zad7 = main.client_by_sum_length(201)
df_zad7

c:\Piotr.K\AGH_STUDIA\semV\Analiza_i_Bazy_Danych\Lab_6\laboratorium-join-agr-sub-piotr0202\main.py:152: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT customer.first_name, customer.last_name, SUM(film.length) AS sum\


,first_name,last_name,sum
0,Brian,Wyman,1265
1,Antonio,Meek,1451
2,Leona,Obrien,1588
3,Katherine,Rivera,1615
4,Tiffany,Jordan,1667
...,...,...,...
594,Wesley,Bull,4808
595,Clara,Shaw,4808
596,Tammy,Sanders,5065
597,Eleanor,Hunt,5360


In [55]:
#8. Oblicz średnią wartość wypożyczenia filmu.
df_zad8 = pd.read_sql('SELECT film.film_id, AVG(payment.amount) FROM payment\
                JOIN rental ON rental.rental_id = payment.rental_id\
                JOIN inventory ON inventory.inventory_id = rental.inventory_id\
                JOIN film ON film.film_id = inventory.film_id\
                GROUP BY film.film_id',con=connection,)
df_zad8

C:\Users\piotr\AppData\Local\Temp\ipykernel_2404\4287079481.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_zad8 = pd.read_sql('SELECT film.film_id, AVG(payment.amount) FROM payment\


,film_id,avg
0,652,2.847143
1,273,1.942381
2,51,4.171818
3,951,1.710000
4,839,1.323333
...,...,...
953,520,2.990000
954,64,1.535455
955,55,3.853636
956,790,1.598696


In [57]:
#9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
df_zad9 = pd.read_sql('SELECT category.name , AVG(film.length) FROM category\
                JOIN film_category ON film_category.category_id = category.category_id\
                JOIN film ON film.film_id = film_category.film_id\
                GROUP BY category.name',con=connection,)
df_zad9

C:\Users\piotr\AppData\Local\Temp\ipykernel_2404\2104958398.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_zad9 = pd.read_sql('SELECT category.name , AVG(film.length) FROM category\


,name,avg
0,Family,114.782609
1,Games,127.836066
2,Animation,111.015152
3,Classics,111.666667
4,Documentary,108.750000
5,New,111.126984
6,Sports,128.202703
7,Children,109.800000
8,Music,113.647059
9,Travel,113.315789


In [75]:
#10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
df_zad10 = pd.read_sql('SELECT category.name, film.title, film.length\
                        FROM film\
                        JOIN film_category ON film.film_id = film_category.film_id\
                        JOIN category ON film_category.category_id = category.category_id\
                        WHERE film.length = (\
                            SELECT MAX(film.length)\
                            FROM film\
                            JOIN film_category ON film.film_id = film_category.film_id\
                            WHERE film_category.category_id = category.category_id\
                        )\
                        ORDER BY category.name',con=connection,)
df_zad10

C:\Users\piotr\AppData\Local\Temp\ipykernel_2404\2595238013.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_zad10 = pd.read_sql('SELECT category.name, film.title, film.length\


,name,title,length
0,Action,Darn Forrester,185
1,Action,Worst Banger,185
2,Animation,Pond Seattle,185
3,Animation,Gangs Pride,185
4,Children,Fury Murder,178
5,Children,Wrong Behavior,178
6,Classics,Conspiracy Spirit,184
7,Comedy,Control Anthem,185
8,Documentary,Young Language,183
9,Documentary,Wife Turn,183


In [85]:
#11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.
df_zad11 = pd.read_sql('SELECT film.title, film.length, category.name\
                        FROM film\
                        JOIN film_category ON film.film_id = film_category.film_id\
                        JOIN category ON film_category.category_id = category.category_id\
                        WHERE film.length = (\
                            SELECT MAX(film.length) FROM film\
                        )',con=connection,)
df_zad11

C:\Users\piotr\AppData\Local\Temp\ipykernel_2404\1812945043.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_zad11 = pd.read_sql('SELECT film.title, film.length, category.name\


,title,length,name
0,Chicago North,185,Games
1,Control Anthem,185,Comedy
2,Darn Forrester,185,Action
3,Gangs Pride,185,Animation
4,Home Pity,185,Music
5,Muscle Bright,185,Travel
6,Pond Seattle,185,Animation
7,Soldiers Evolution,185,Sci-Fi
8,Sweet Brotherhood,185,Travel
9,Worst Banger,185,Action
